In [1]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd

In [2]:
df = pd.read_csv("../data/RHH_Events_Data.csv")
venues = pd.read_csv("../data/RHH_Venues.csv")
df = df.join(venues.set_index('Venue'), on='Venue', how='outer').drop(['Visits', 'Closest Metros', 'Address'], axis=1)

In [3]:
def get_weather_data(row, time=18): # Time defaults to 6pm
    #print(row)
    HH_date = datetime.strptime(row['Date'], '%m/%d/%Y')
    HH_date = HH_date.replace(hour=time)

    location = Point(row['lat'], row['lon'], 70)


    # Get daily data for 2018
    data = Hourly(location, HH_date, HH_date)
    data = data.fetch()
    return data.loc[:,['temp', 'rhum', 'prcp', 'snow', 'wspd', 'tsun']].iloc[0]

In [5]:
df[['Temperature','Humidity','Precipitation_mm','Snow_mm','Wind_Speed','Total_Sunshine']] = df.apply(get_weather_data, axis=1)

In [6]:
df = df.drop(['lat', 'lon'], axis=1)

In [8]:
df['Temperature'] = df['Temperature'].apply(lambda celsius: (celsius * 1.8) + 32)

In [9]:
df

,Unnamed: 0,Number,Date,Venue,Venue_2,Official,Temperature,Relative_Humidity,Precipitation_mm,Snow_mm,Wind_Speed,Total_Sunshine
0,0,262,5/20/2022,Electric Cool Aid,NaN,True,83.48,59.0,0.0,NaN,17.0,NaN
1,9,-9,3/18/2022,Electric Cool Aid,NaN,False,71.24,53.0,0.0,NaN,14.8,NaN
2,26,249,10/15/2021,Electric Cool Aid,NaN,True,78.26,62.0,0.0,NaN,14.8,NaN
3,1,261,5/13/2022,Atlas Brew Works,NaN,True,70.34,79.0,0.0,NaN,13.0,NaN
4,2,260,5/6/2022,Smoke & Barrel,NaN,True,61.88,86.0,1.4,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
270,266,9,8/28/2015,Stetson's,NaN,True,81.32,44.0,0.0,NaN,7.6,NaN
271,267,8,8/21/2015,Stetson's,NaN,True,83.30,38.0,0.0,NaN,11.2,NaN
272,268,7,8/14/2015,Stetson's,NaN,True,84.20,39.0,0.0,NaN,9.4,NaN
273,270,5,7/31/2015,Stetson's,NaN,True,90.32,36.0,0.0,NaN,9.4,NaN


In [95]:
df.to_csv("../data/RHH_Events_Data.csv")